# 태스크 4: 대화형 인터페이스 - Llama 3 및 Titan Premier LLM과 채팅

이 노트북에서는 Amazon Bedrock의 llama3-8b-instruct 및 titan-text-premier 파운데이션 모델(FM)을 사용하여 챗봇을 구축합니다.

챗봇 및 가상 도우미와 같은 대화형 인터페이스를 통해 고객의 사용자 경험을 강화할 수 있습니다. 챗봇은 자연어 처리(NLP) 및 기계 학습(ML) 알고리즘을 사용하여 사용자 쿼리를 이해하고 응답합니다. 고객 서비스, 판매, 전자 상거래 등 다양한 애플리케이션에서 챗봇을 사용하여 사용자에게 빠르고 효율적인 응답을 제공할 수 있습니다. 사용자가 웹사이트, 소셜미디어 플랫폼, 메시징 앱 등 다양한 채널을 통해 액세스할 수 있습니다.

- **챗봇(기본)** - FM 모델을 사용하는 제로샷 챗봇
- **프롬프트를 사용하는 챗봇** - 템플릿(Langchain) - 프롬프트 템플릿에 제공된 일부 컨텍스트를 사용하는 챗봇
- **가상 사용자를 사용하는 챗봇** - 정의된 역할(예: 진로 담당 코치 및 인간 상호 작용)이 있는 챗봇
- **컨텍스트 인식 챗봇** - 임베딩을 생성하여 외부 파일을 통해 컨텍스트 전달

## Amazon Bedrock으로 챗봇을 구축하기 위한 LangChain 프레임워크

챗봇과 같은 인터랙티브 인터페이스에서는 단기 및 장기 수준에서 이전 상호작용을 기억하는 것이 매우 중요합니다.

LangChain 프레임워크에서는 메모리 구성 요소를 두 가지 형태로 제공합니다. 첫째, LangChain은 이전 채팅 메시지를 관리하고 조작하기 위한 도우미 유틸리티를 제공합니다. 유틸리티는 모듈식으로 설계되었습니다. 둘째, LangChain에서는 다양한 유형의 추상화를 쉽게 정의하고 상호 작용하여 강력한 챗봇을 쉽게 구축할 수 있도록 이러한 유틸리티를 체인에 통합하는 쉬운 방법을 제공합니다.

## 컨텍스트를 사용하여 챗봇 구축 - 핵심 요소

컨텍스트 인식 챗봇을 구축하는 첫 번째 프로세스는 컨텍스트에 대한 임베딩을 생성하는 것입니다. 일반적으로, 임베딩 모델을 통해 실행하여 임베딩을 생성하면 벡터 저장소에 저장되는 수집 프로세스가 있습니다. 이 노트북에서는 여기에 Titan Embeddings 모델을 사용합니다. 두 번째 프로세스는 사용자 요청 오케스트레이션, 상호작용, 호출 및 결과 반환입니다. 여기에는 사용자 요청을 오케스트레이션하고, 필요한 모델/구성 요소와 상호 작용하여 정보를 수집하고, 챗봇을 호출하여 응답을 공식화한 다음에 챗봇의 응답을 다시 사용자에게 반환하는 것이 포함됩니다.

## 태스크 4.1: 환경 설정

이 태스크에서는 환경을 설정합니다.

In [ ]:
#ignore warnings and create a service client by name using the default session.
import json
import os
import sys
import warnings

import boto3

warnings.filterwarnings('ignore')
module_path = ".."
sys.path.append(os.path.abspath(module_path))
bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))


In [ ]:
# format instructions into a conversational prompt
from typing import Dict, List

def format_instructions(instructions: List[Dict[str, str]]) -> List[str]:
    """Format instructions where conversation roles must alternate system/user/assistant/user/assistant/..."""
    prompt: List[str] = []
    for instruction in instructions:
        if instruction["role"] == "system":
            prompt.extend(["<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n", (instruction["content"]).strip(), " <|eot_id|>"])
        elif instruction["role"] == "user":
            prompt.extend(["<|start_header_id|>user<|end_header_id|>\n", (instruction["content"]).strip(), " <|eot_id|>"])
        else:
            raise ValueError(f"Invalid role: {instruction['role']}. Role must be either 'user' or 'system'.")
    prompt.extend(["<|start_header_id|>assistant<|end_header_id|>\n"])
    return "".join(prompt)

## 태스크 4.2: LangChain의 채팅 히스토리를 사용하여 대화 시작

이 태스크에서는 챗봇이 사용자와의 여러 상호 작용을 통해 대화의 컨텍스트를 전달할 수 있도록 합니다. 챗봇이 시간이 지나도 의미 있고 일관된 대화를 유지하려면 대화형 메모리가 있어야 합니다.

LangChain의 InMemoryChatMessageHistory 클래스를 기반으로 구축하여 대화형 메모리 기능을 구현합니다. 이 객체에서는 사용자와 챗봇 간 대화를 저장하며, 챗봇 에이전트가 이전 대화의 컨텍스트를 활용할 수 있도록 히스토리가 제공됩니다.

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **참고:** 모델 출력은 비결정적입니다.

In [ ]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_aws import ChatBedrock

chat_model=ChatBedrock(
    model_id="meta.llama3-8b-instruct-v1:0" , 
    client=bedrock_client)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Answer the following questions as best you can."),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

history = InMemoryChatMessageHistory()


def get_history():
    return history


chain = prompt | chat_model | StrOutputParser()

wrapped_chain = RunnableWithMessageHistory(
    chain,
    get_history,
    history_messages_key="chat_history",
)
query="how are you?"
response=wrapped_chain.invoke({"input": query})
# Printing history to see the history being built out. 
print(history)
# For the rest of the conversation, the output will only include response

### 새 질문

모델에서 초기 메시지로 응답했습니다. 이제 몇 가지를 질문합니다.

In [ ]:
#new questions
instructions = [{"role": "user", "content": "Give me a few tips on how to start a new garden."}]
response=wrapped_chain.invoke({"input": format_instructions(instructions)})
print(response)

### 질문을 바탕으로 구축

이제 garden이라는 단어를 언급하지 않고 질문을 제시하여 모델이 이전 대화를 이해할 수 있는지 확인하겠습니다.

In [ ]:
# build on the questions
instructions = [{"role": "user", "content": "bugs"}]
response=wrapped_chain.invoke({"input": format_instructions(instructions)})
print(response)

### 이 대화 마무리

In [ ]:
# finishing the conversation
instructions = [{"role": "user", "content": "That's all, thank you!"}]
response=wrapped_chain.invoke({"input": format_instructions(instructions)})
print(response)

## 태스크 4.3: 프롬프트 템플릿을 사용하는 챗봇(Langchain)

이 태스크에서는 이 입력의 구성을 담당하는 기본 PromptTemplate을 사용합니다. LangChain은 몇 가지 클래스 및 함수를 제공하여 프롬프트를 쉽게 생성하고 사용할 수 있도록 합니다.

In [ ]:
#  prompt for a conversational agent
def format_prompt(actor:str, input:str):
    formatted_prompt: List[str] = []
    if actor == "system":
        prompt_template="""<|begin_of_text|><|start_header_id|>{actor}<|end_header_id|>\n{input}<|eot_id|>"""
    elif actor == "user":
        prompt_template="""<|start_header_id|>{actor}<|end_header_id|>\n{input}<|eot_id|>"""
    else:
        raise ValueError(f"Invalid role: {actor}. Role must be either 'user' or 'system'.")   
    prompt = PromptTemplate.from_template(prompt_template)     
    formatted_prompt.extend(prompt.format(actor=actor,input=input))
    formatted_prompt.extend(["<|start_header_id|>assistant<|end_header_id|>\n"])
    return "".join(formatted_prompt)

In [ ]:
# chat user experience
import ipywidgets as ipw
from IPython.display import display, clear_output

class ChatUX:
    """ A chat UX using IPWidgets
    """
    def __init__(self, qa, retrievalChain = False):
        self.qa = qa
        self.name = None
        self.b=None
        self.retrievalChain = retrievalChain
        self.out = ipw.Output()


    def start_chat(self):
        print("Starting chat bot")
        display(self.out)
        self.chat(None)


    def chat(self, _):
        if self.name is None:
            prompt = ""
        else: 
            prompt = self.name.value
        if 'q' == prompt or 'quit' == prompt or 'Q' == prompt:
            with self.out:
                print("Thank you , that was a nice chat !!")
            return
        elif len(prompt) > 0:
            with self.out:
                thinking = ipw.Label(value="Thinking...")
                display(thinking)
                try:
                    if self.retrievalChain:
                        response = self.qa.invoke({"input": prompt})
                        result=response['answer']
                    else:
                        instructions = [{"role": "user", "content": prompt}]
                        #result = self.qa.invoke({'input': format_prompt("user",prompt)}) #, 'history':chat_history})
                        result = self.qa.invoke({"input": format_instructions(instructions)})
                except:
                    result = "No answer"
                thinking.value=""
                print(f"AI:{result}")
                self.name.disabled = True
                self.b.disabled = True
                self.name = None

        if self.name is None:
            with self.out:
                self.name = ipw.Text(description="You:", placeholder='q to quit')
                self.b = ipw.Button(description="Send")
                self.b.on_click(self.chat)
                display(ipw.Box(children=(self.name, self.b)))

그런 다음에 채팅을 시작합니다.

In [ ]:
# start chat
history = InMemoryChatMessageHistory() #reset chat history
chat = ChatUX(wrapped_chain)
chat.start_chat()

In [ ]:
print(history)

## 태스크 4.4: 페르소나 챗봇

이 태스크에서는 인공 지능(AI) 도우미가 커리어 코치 역할을 합니다. 시스템 메시지를 사용하여 페르소나(또는 역할)에 대한 정보를 챗봇에 제공할 수 있습니다. InMemoryChatMessageHistory 클래스를 계속 활용하여 대화 컨텍스트를 유지 관리합니다.

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", " You will be acting as a career coach. Your goal is to give career advice to users. For questions that are not career related, don't provide advice. Say, I don't know."),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

history = InMemoryChatMessageHistory() # reset history

chain = prompt | chat_model | StrOutputParser()

wrapped_chain = RunnableWithMessageHistory(
    chain,
    get_history,
    history_messages_key="career_chat_history",
)

response=wrapped_chain.invoke({"input": "What are the career options in AI?"})
print(response)

In [ ]:
response=wrapped_chain.invoke({"input": "How to fix my car?"})
print(response)

In [ ]:
print(history)

이제 이 페르소나의 전문 분야가 아닌 질문을 제시합니다. 모델은 해당 질문에 답변하지 않아야 하며 그 이유를 제시해야 합니다.

## 태스크 4.5: 컨텍스트를 사용하는 챗봇

이 태스크에서는 챗봇에 전달되었던 컨텍스트를 기반으로 질문에 답변하도록 챗봇에 요청합니다. CSV 파일을 가져오고 Titan 임베딩 모델을 사용하여 해당 컨텍스트를 나타내는 벡터를 생성합니다. 이 벡터는 Facebook AI Similarity Search(FAISS)에 저장됩니다. 챗봇이 질문을 받으면 이 벡터를 챗봇에 다시 전달하고 벡터를 사용하여 답변을 검색하도록 합니다.

### Titan 임베딩 모델

임베딩에서는 단어, 구문 또는 다른 개별 항목을 연속 벡터 공간의 벡터로 나타냅니다. 이를 통해 기계 학습 모델은 이러한 표현에 대해 수학 연산을 수행하고 이러한 표현 간의 의미 체계 관계를 캡처할 수 있습니다.

검색 증강 생성(RAG) [문서 검색 기능](https://labelbox.com/blog/how-vector-similarity-search-works/)에 임베딩을 사용합니다.

In [ ]:
# model configuration
from langchain_aws.embeddings import BedrockEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate

br_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1", client=bedrock_client)

### VectorStore로서의 FAISS

검색에 임베딩을 사용하려면 벡터 유사성 검색을 효율적으로 수행할 수 있는 스토어가 필요합니다. 이 노트북에서는 인 메모리 스토어인 FAISS를 사용합니다. 벡터를 영구적으로 저장하려면 Amazon Bedrock, pgVector, Pinecone, Weaviate 또는 Chroma용 지식 기반을 사용할 수 있습니다.

LangChain VectorStore API는 [여기](https://python.langchain.com/v0.2/docs/integrations/vectorstores/)에서 제공됩니다.

In [ ]:
# vector store
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

loader = CSVLoader("../rag_data/Amazon_SageMaker_FAQs.csv") # --- > 219 docs with 400 chars
documents_aws = loader.load() #
print(f"documents:loaded:size={len(documents_aws)}")

docs = CharacterTextSplitter(chunk_size=2000, chunk_overlap=400, separator=",").split_documents(documents_aws)

print(f"Documents:after split and chunking size={len(docs)}")
vectorstore_faiss_aws = None
try:
    
    vectorstore_faiss_aws = FAISS.from_documents(
        documents=docs,
        embedding = br_embeddings, 
        #**k_args
    )

    print(f"vectorstore_faiss_aws:created={vectorstore_faiss_aws}::")

except ValueError as error:
    if  "AccessDeniedException" in str(error):
        print(f"\x1b[41m{error}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error

### 빠른 로우 코드 테스트 실행 

LangChain에서 제공하는 Wrapper 클래스를 사용하여 벡터 데이터베이스 스토어를 쿼리하고 관련 문서를 반환할 수 있습니다. 그러면 모든 기본값을 사용하여 QA 체인이 실행됩니다.

In [ ]:
chat_llm=ChatBedrock(
    model_id="amazon.titan-text-premier-v1:0" , 
    client=bedrock_client)
# wrapper store faiss
wrapper_store_faiss = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss_aws)
print(wrapper_store_faiss.query("R in SageMaker", llm=chat_llm))

### 챗봇 애플리케이션

챗봇의 경우 컨텍스트 관리, 히스토리, 벡터 스토어 및 기타 많은 구성 요소가 필요합니다. 컨텍스트가 지원되는 검색 증강 생성(RAG) 사슬을 구축하여 시작합니다.

여기에서는 **create_stuff_documents_chain** 및 **create_retrieval_chain** 함수를 사용합니다.

### RAG에 사용되는 파라미터 및 함수

- **Retriever:** `VectorStore`에서 지원되는 `VectorStoreRetriever`를 사용했습니다. 텍스트를 검색하려면 `"similarity"` 또는 `"mmr"`의 두 가지 검색 유형 중에서 선택할 수 있습니다. `search_type="similarity"`에서는 검색기 객체에서 유사성 검색을 사용합니다. 여기에서는 질문 벡터와 가장 비슷한 텍스트 청크 벡터를 선택합니다.

- **create_stuff_documents_chain**에서는 검색한 컨텍스트가 프롬프트와 LLM에 제공되는 방식이 지정됩니다. 검색한 문서는 프롬프트에 요약이나 기타 처리 없이 프롬프트에 "stuffed"됩니다.

- **create_retrieval_chain**에서는 검색 단계가 추가되고 최종 답변과 함께 제공되는 검색된 컨텍스트가 체인을 통해 자동으로 입력됩니다. 

제시된 질문이 컨텍스트 범위를 벗어나면 모델에서는 답변을 모른다고 회신합니다.

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

retriever=vectorstore_faiss_aws.as_retriever()
question_answer_chain = create_stuff_documents_chain(chat_llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": "What is sagemaker?"})
print(response) # shows the document chunks consulted to come up with the answer

그런 다음에 채팅을 시작합니다.

In [ ]:
chat = ChatUX(rag_chain, retrievalChain=True)
chat.start_chat()  # Only answers will be shown here, and not the citations


Titan LLM을 이용하여 다음과 같은 패턴의 대화형 인터페이스를 생성했습니다.

- 챗봇(기본 - 컨텍스트 없음)
- 프롬프트 템플릿을 사용하는 챗봇(Langchain)
- 가상 사용자를 사용하는 챗봇
- 컨텍스트를 사용하는 챗봇

### 직접 해보기

- 프롬프트를 특정 사용 사례로 변경하고 다양한 모델의 출력을 평가하십시오.
- 토큰 길이를 조작하면서 서비스의 지연 시간 및 응답성을 이해해 보십시오.
- 다양한 프롬프트 엔지니어링 원칙을 적용하여 더 나은 출력을 얻을 수 있습니다.

### 정리

이 노트북을 완료했습니다. 실습의 다음 부분으로 이동하려면 다음을 수행합니다.

- 이 노트북 파일을 닫고 **태스크 5**를 계속 진행합니다.